In [1]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
from PIL import Image

In [2]:
# Run locally
folder_link = ".."

In [20]:
train_data = []
train_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"]:
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/training_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/validation_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/training_output.csv"))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/validation_output.csv"))

train_data = np.concatenate(train_data)
print(train_data.shape)

test_data = []
test_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"] + ["bonk", "floki", "pepe", "wif"]:
    test_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/testing_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    test_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/testing_output.csv"))

test_data = np.concatenate(test_data)
print(test_data.shape)

(280437, 54, 72)
(70297, 54, 72)


In [21]:
def normalize(df):
    df["Max24"] = df["High"].shift(1).rolling(window = 24).max()
    df["Max24"] = df["Max24"].fillna(df.loc[24, "Max24"])
    df["Min24"] = df["Low"].shift(1).rolling(window = 24).min()
    df["Min24"] = df["Min24"].fillna(df.loc[24, "Min24"])
    df["Scale"] = np.log(df["Max24"]) - np.log(df["Min24"])
    df["NormalizedLR"] = df["LogReturns"] / df["Scale"]
    df = df.drop(columns=["Volume", "Close time", "Quote asset volume", 'Number of trades', "Taker buy base asset volume", "Taker buy quote asset volume",
                          "High", "Low", "Max24", "Min24", "Scale"], axis = 1)
    return df

for i in range(len(train_output_arr)):
    train_output_arr[i] = normalize(train_output_arr[i])
for i in range(len(test_output_arr)):
    test_output_arr[i] = normalize(test_output_arr[i])

train_output = pd.concat(train_output_arr)
test_output = pd.concat(test_output_arr)

In [22]:
train_output

,Open time,LogReturns,NormalizedLR
0,1609545600000,-0.019493,-0.206317
1,1609549200000,0.010832,0.114642
2,1609552800000,0.004025,0.042597
3,1609556400000,0.004180,0.044241
4,1609560000000,-0.004065,-0.043026
...,...,...,...
2154,1711908000000,0.002394,0.105351
2155,1711911600000,0.002229,0.113083
2156,1711915200000,-0.002070,-0.104998
2157,1711918800000,-0.000638,-0.032351


In [ ]:
# Save train_data
np.savez_compressed("train_data", train_data, allow_pickle = False)
train_output.to_csv("train_output.csv", index = False)

# Save test_data
np.savez_compressed("test_data", test_data, allow_pickle = False)
test_output.to_csv("test_output.csv", index = False)